<a href="https://colab.research.google.com/github/Tilakraj-B/cardiac-disease/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import zipfile
from google.colab import drive
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate
from tensorflow.keras.models import Model

In [33]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
dataset_path = "/content/drive/MyDrive/acdc_dataset.zip"  # Update this path
extract_folder = "/content/acdc_data"

In [35]:
# Step 3: Extract the dataset if not already extracted
if not os.path.exists(extract_folder):
    print("Extracting dataset...")
    with zipfile.ZipFile(dataset_path, "r") as zip_ref:
        zip_ref.extractall(extract_folder)
    print("Extraction complete.")
else:
    print("Dataset already extracted.")


Dataset already extracted.


In [36]:
def load_nii_file(filepath):
    img = nib.load(filepath)
    data = img.get_fdata()
    return data


In [37]:
# Step 5: Load and visualize a sample image (Modify the path based on extracted files)
base_dir = os.path.join(extract_folder,"database")
training_folder = os.path.join(extract_folder,"database","training")
testing_folder = os.path.join(extract_folder,"database","testing")
train_patients = [f'patient{i:03d}' for i in range(1, 50)]
test_patients = [f'patient{i:03d}' for i in range(1, 50)]

# info_file_path = os.path.join(patient_folder, "Info.cfg")

# if os.path.exists(info_file_path):
#     with open(info_file_path, "r") as file:
#         info_content = file.read()
#     print("Contents of Info.cfg:\n")
#     print(info_content)
# else:
#     print("Info.cfg not found. Check the path!")



In [38]:
# Display all the images of a patient

def display_nifti_slices(nifti_path):
    nifti_img = nib.load(nifti_path)  # Load NIfTI file
    image_data = nifti_img.get_fdata()  # Convert to NumPy array

    print(f"\nDisplaying slices for {os.path.basename(nifti_path)}")
    print(f"Image shape: {image_data.shape}")  # Print shape

    dim = image_data.ndim  # Get number of dimensions
    if dim == 2:  # Case: 2D Image
        plt.figure(figsize=(6, 6))
        plt.imshow(image_data, cmap="gray")
        plt.title(f"{os.path.basename(nifti_path)} - 2D Image")
        plt.axis("off")
        plt.show()

    elif dim == 3:  # Case: 3D Image (H, W, D)
        num_slices = image_data.shape[2]
        for i in range(num_slices):
            plt.figure(figsize=(6, 6))
            plt.imshow(image_data[:, :, i], cmap="gray")
            plt.title(f"{os.path.basename(nifti_path)} - Slice {i+1}/{num_slices}")
            plt.axis("off")
            plt.show()

    elif dim == 4:  # Case: 4D Image (H, W, D, T)
        num_slices = image_data.shape[2]
        num_timeframes = image_data.shape[3]
        for t in range(num_timeframes):  # Loop over time
            for i in range(num_slices):  # Loop over depth
                plt.figure(figsize=(6, 6))
                plt.imshow(image_data[:, :, i, t], cmap="gray")
                plt.title(f"{os.path.basename(nifti_path)} - Time {t+1}/{num_timeframes} - Slice {i+1}/{num_slices}")
                plt.axis("off")
                plt.show()

    else:
        print(f"Unsupported image dimension: {dim}")

In [39]:
# # displaying dimension and shape of the imagedatabase
# for i in range(1,100):
#   patient_folder = os.path.join(training_folder, f"patient{i:03d}")
#   for filename in sorted(os.listdir(patient_folder)):
#     if filename.endswith(".nii"):
#       nifti_path = os.path.join(patient_folder, filename)
#       # display_nifti_slices(nifti_path)
#       nifti_img = nib.load(nifti_path)  # Load NIfTI file
#       image_data = nifti_img.get_fdata()  # Convert to NumPy array
#       dim = image_data.ndim
#       # print(filename + " : " + str(dim) + " dimensions" + str(image_data.shape))


In [40]:
def load_patient_data(patient_folder):
    # Get all .nii files in the folder
    files = [f for f in os.listdir(patient_folder) if f.endswith('.nii') and not f.endswith('4d.nii')]

    # Separate image and ground truth files
    image_files = [f for f in files if not f.endswith('_gt.nii')]
    gt_files = [f for f in files if f.endswith('_gt.nii')]

    images = []
    masks = []

    for img_file, gt_file in zip(sorted(image_files), sorted(gt_files)):
        img_data = load_nii_file(os.path.join(patient_folder, img_file))
        gt_data = load_nii_file(os.path.join(patient_folder, gt_file))

        #iterating through each slice in the model
        for slice_idx in range(img_data.shape[2]):
            img_slice = img_data[..., slice_idx]
            gt_slice = gt_data[..., slice_idx]

            # Normalize and resize if needed
            img_slice = (img_slice - img_slice.min()) / (img_slice.max() - img_slice.min())
            gt_slice = (gt_slice > 0).astype(np.float32)  # Binarize if needed

            # Resize to make dimensions divisible by 16 for U-Net (optional)
            img_slice = tf.image.resize(img_slice[..., np.newaxis], [224, 256])
            gt_slice = tf.image.resize(gt_slice[..., np.newaxis], [224, 256])

            images.append(img_slice.numpy())
            masks.append(gt_slice.numpy())

    return np.array(images), np.array(masks)




In [41]:
def load_dataset(root_folder, patient_range):
    all_images = []
    all_masks = []

    for patient_id in patient_range:
        patient_folder = os.path.join(root_folder, f"patient{patient_id:03d}")
        if os.path.exists(patient_folder):
            images, masks = load_patient_data(patient_folder)
            all_images.extend(images)
            all_masks.extend(masks)

    return np.array(all_images), np.array(all_masks)


In [42]:
def unet_model(input_size=(224, 256, 1)):
    inputs = Input(input_size)

    # Downsample path
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    # Bottom
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)

    # Upsample path
    up6 = Conv2D(512, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(drop5))
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = Conv2D(256, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv6))
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = Conv2D(128, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv7))
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = Conv2D(64, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv8))
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)

    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=outputs)

    return model

In [43]:
def train_model():
    # Load training data
    train_images, train_masks = load_dataset(training_folder, range(1, 101))

    # Split into training and validation
    X_train, X_val, y_train, y_val = train_test_split(
        train_images, train_masks, test_size=0.2, random_state=42
    )

    # Create model
    model = unet_model()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.IoU(num_classes=2, target_class_ids=[1])])

    # Callbacks
    callbacks = [
        tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True),
        tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_loss')
    ]

    # Train
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=16,
        epochs=100,
        callbacks=callbacks
    )

    return model, history

In [44]:
def evaluate_model(model):
    # Load test data
    test_images, test_masks = load_dataset(testing_folder, range(101, 151))

    # Evaluate
    results = model.evaluate(test_images, test_masks)
    print(f"Test Loss: {results[0]}, Test Accuracy: {results[1]}, Test IoU: {results[2]}")

    # Optionally visualize some predictions
    import matplotlib.pyplot as plt

    preds = model.predict(test_images[:5])

    plt.figure(figsize=(15, 5))
    for i in range(10):
        plt.subplot(3, 5, i+1)
        plt.imshow(test_images[i].squeeze(), cmap='gray')
        plt.title('Input')
        plt.axis('off')

        plt.subplot(3, 5, i+6)
        plt.imshow(test_masks[i].squeeze(), cmap='gray')
        plt.title('Ground Truth')
        plt.axis('off')

        plt.subplot(3, 5, i+11)
        plt.imshow(preds[i].squeeze() > 0.5, cmap='gray')
        plt.title('Prediction')
        plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
if __name__ == "__main__":
    # Train the model
    model, history = train_model()

    # Evaluate on test set
    evaluate_model(model)

    # Optionally save the model
    model.save('unet_mri_segmentation.keras')

Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 113s/step - accuracy: 0.7841 - io_u_3: 0.0000e+00 - loss: 11.9001  

19/19 ━━━━━━━━━━━━━━━━━━━━ 2372s 125s/step - accuracy: 0.7899 - io_u_3: 0.0000e+00 - loss: 11.6650 - val_accuracy: 0.9523 - val_io_u_3: 0.0000e+00 - val_loss: 0.2341
Epoch 2/10
 3/19 ━━━━━━━━━━━━━━━━━━━━ 32:07 120s/step - accuracy: 0.9547 - io_u_3: 0.0000e+00 - loss: 0.2239